In [1]:

from ldap3 import Server, Connection, ALL, SIMPLE

server = Server("ldaps://CIC-VDC1.c-i-c.co.uk", use_ssl=True, get_info=ALL)

conn = Connection(
    server,
    user="wingsan.chow@c-i-c.co.uk",   # UPN login
    password="P@$$3399#2021jun",
    authentication=SIMPLE,
    auto_bind=True
)

print("Connected:", conn.bound)


Connected: True


In [2]:
from ldap3 import Server, Connection, ALL, SIMPLE

# Internal AD DNS domain (the LDAP server)
LDAP_HOST = "ldaps://CIC-VDC1.c-i-c.co.uk"

# Credentials in UPN form
USERNAME = "wingsan.chow@c-i-c.co.uk"
PASSWORD = "P@$$3399#2021jun"

# Base DN derived from the internal AD domain
BASE_DN = "DC=c-i-c,DC=co,DC=uk"

# Connect securely using LDAPS + SIMPLE BIND
server = Server(LDAP_HOST, use_ssl=True, get_info=ALL)
conn = Connection(
    server,
    user=USERNAME,
    password=PASSWORD,
    authentication=SIMPLE,
    auto_bind=True
)

print("Connected:", conn.bound)

# Search
conn.search(
    BASE_DN,
    "(objectClass=group)",
    attributes=["cn"]     # request specific attribute
)

# Safe parsing of results
groups = []
for entry in conn.entries:
    data = entry.entry_attributes_as_dict
    if "cn" in data:
        groups.append(data["cn"][0])   # cn is always a list
    else:
        # avoid crash; print what AD returned
        print("Entry missing CN:", entry.entry_dn)

#print(groups)



Connected: True


In [3]:
print(len(groups))

646


In [4]:
import pandas as pd

grp = pd.DataFrame(groups)
grp.head(200)

,0
0,Organization Management
1,Public Folder Management
2,Recipient Management
3,View-Only Organization Management
4,UM Management
...,...
195,COOP
196,Payroll
197,UAT3_Great_Plains_Interface
198,GRG SQL DB Read Only Access


In [5]:
from ldap3 import Server, Connection, ALL, SIMPLE, SUBTREE

# Internal AD DNS domain (the LDAP server)
LDAP_HOST = "ldaps://CIC-VDC1.c-i-c.co.uk"

# Credentials in UPN form
USERNAME = "wingsan.chow@c-i-c.co.uk"
PASSWORD = "P@$$3399#2021jun"

# Base DN derived from the internal AD domain
BASE_DN = "DC=c-i-c,DC=co,DC=uk"

# Connect securely using LDAPS + SIMPLE BIND
server = Server(LDAP_HOST, use_ssl=True, get_info=ALL)
conn = Connection(
    server,
    user=USERNAME,
    password=PASSWORD,
    authentication=SIMPLE,
    auto_bind=True
)

print("Connected:", conn.bound)

attrs = ["cn", "distinguishedName", "member"]

conn.search(
    BASE_DN,
    "(&(objectClass=group)(cn=*))",
    attributes=attrs
)

records = []
for e in conn.entries:
    d = e.entry_attributes_as_dict
    group_dn = e.entry_dn
    group_cn = (d.get("cn") or [None])[0]
    members = d.get("member") or []   # this is a list
    if not members:
        records.append({"group_cn": group_cn, "group_dn": group_dn, "member_dn": None})
    else:
        for m in members:
            records.append({"group_cn": group_cn, "group_dn": group_dn, "member_dn": m})

df_membership = pd.DataFrame(records)
df_membership.head(100)
# df_membership.to_csv("ad_group_membership.csv", index=False)



# 2. First search: find all groups and collect their members
conn.search(
    BASE_DN,
    "(&(objectClass=group)(cn=*))",
    attributes=["cn", "member"]
)

group_rows = []   # Will store: group_cn, group_dn, member_dn
all_member_dns = set()

for entry in conn.entries:
    d = entry.entry_attributes_as_dict
    group_cn = d.get("cn", [None])[0]
    members = d.get("member", []) or []

    if not members:
        group_rows.append({
            "group_cn": group_cn,
            "group_dn": entry.entry_dn,
            "member_dn": None
        })
    else:
        for m in members:
            group_rows.append({
                "group_cn": group_cn,
                "group_dn": entry.entry_dn,
                "member_dn": m
            })
            all_member_dns.add(m)

df_groups = pd.DataFrame(group_rows)

# 3. Second search: lookup each user DN → email + UPN + sAMAccountName
member_details = []

for dn in all_member_dns:
    conn.search(
        search_base=dn,
        search_filter="(objectClass=user)",
        search_scope=SUBTREE,
        attributes=["mail", "userPrincipalName", "sAMAccountName", "displayName", "objectGUID", "userAccountControl"]
    )

    if conn.entries:
        e = conn.entries[0].entry_attributes_as_dict
        member_details.append({
            "member_dn": dn,
            "email": (e.get("mail") or [None])[0],
            "userPrincipalName": (e.get("userPrincipalName") or [None])[0],
            "sAMAccountName": (e.get("sAMAccountName") or [None])[0],
            "displayName": (e.get("displayName") or [None])[0],
            "objectGUID": (e.get("objectGUID") or [None])[0],
            "userAccountControl": (e.get("userAccountControl") or [None])[0],
        })
    else:
        member_details.append({
            "member_dn": dn,
            "email": None,
            "userPrincipalName": None,
            "sAMAccountName": None,
            "displayName": None,
            "objectGUID": None,
            "userAccountControl": None
        })

df_members = pd.DataFrame(member_details)

# 4. Merge group info with user info
df_final = df_groups.merge(df_members, on="member_dn", how="left")



Connected: True


In [7]:
grp.to_csv("ad_groups.csv", index=False, encoding="utf-8-sig")
grp.to_excel("ad_groups.xlsx", index=False)
#df_membership.to_parquet("ad_group_membership.parquet", index=False)  # compact & fast
df_membership.to_csv("ad_grp_user.csv", index=False, encoding="utf-8-sig")
df_final.to_csv("ad_groups_user_detail.csv", index=False, encoding="utf-8-sig")
df_final.to_excel("ad_groups_user_detail.xlsx", index=False)